In [14]:
import pandas as pd
import numpy as np

# 데이터 열기

In [15]:
df_movie = pd.read_csv('./data/x_train_valid.csv')
df_oscars = pd.read_csv('./data/oscars_2020_2025.csv')
df_tmbd_top = pd.read_csv('./data/tmbd_top10_2020_2025.csv')

In [16]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing
0,Stepmom's Desire,2020-05-29,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,ko,unknown,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0
1,Pizza Dare 1,2020-04-10,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,ko,unknown,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0
2,Bosomy Mom,2020-05-06,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,ko,unknown,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0
3,Sonic the Hedgehog,2020-02-12,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,en,"Original Film, Blur Studio, Marza Animation Pl...","Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0
4,12,2020-09-04,movie,15.0,1.0,1.0,NaN,Salih Hashwan,unknown,SA,ar,unknown,not there yet,"short film, short film",5.481,53,not there yet,1,0,1


In [17]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,NaN
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,NaN
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,NaN
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,NaN


In [18]:
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020-11-25,8.500,165
1,US,United States,Gabriel's Inferno,2020-05-29,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020-07-31,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020-11-19,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020-12-26,8.400,83


In [19]:
print("df_movie.columns:", df_movie.columns.tolist())
print("df_oscars.columns:", df_oscars.columns.tolist())
print("df_tmbd_top.columns:", df_tmbd_top.columns.tolist())

df_movie.columns: ['title', 'release_date', 'type', 'runtime', 'season_count', 'episode_count', 'genre', 'director', 'cast', 'country', 'language', 'production_company', 'synopsis', 'tags', 'imdb_rating', 'imdb_review_count', 'text_input', 'production_company_is_missing', 'director_is_missing', 'cast_is_missing']
df_oscars.columns: ['Ceremony', 'Year', 'Class', 'Category', 'Film', 'Name', 'Winner']
df_tmbd_top.columns: ['country', 'country_name', 'title', 'release_date', 'vote_average', 'vote_count']


# df_movie

In [20]:
df_movie.describe()

,runtime,season_count,episode_count,imdb_rating,imdb_review_count,production_company_is_missing,director_is_missing,cast_is_missing
count,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000,66300.000000
mean,71.011810,1.081267,5.827481,4.183419,45.610483,0.284087,0.162926,0.050015
std,56.308355,0.543860,22.268006,3.337211,344.562730,0.450982,0.369301,0.217978
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,79.000000,1.000000,1.000000,5.333000,2.000000,0.000000,0.000000,0.000000
75%,97.000000,1.000000,1.000000,7.000000,8.000000,1.000000,0.000000,0.000000
max,6250.000000,62.000000,1204.000000,10.000000,20819.000000,1.000000,1.000000,1.000000


<strong>- imdb_rating * imdb_review_count 하여 상위 20%에 1 부여</strong>

In [21]:
df_movie["imdb_score"] = df_movie["imdb_rating"] * np.log1p(df_movie["imdb_review_count"])
threshold = df_movie["imdb_score"].quantile(0.8)

df_movie["success_imdb"] = (df_movie["imdb_score"] >= threshold).astype(int)

In [22]:
df_movie["success_imdb"].value_counts()

success_imdb
0    53037
1    13263
Name: count, dtype: int64

# concat oscars


| 클래스                                                 | 의미            | 점수   |
| --------------------------------------------------- | ------------- | ---- |
| 수상 (`Winner == "True"`)                             | 실제 오스카 수상작/인물 | `2`점 |
| 후보 (`Class == "Nomination"` and `Winner != "True"`) | 수상은 못 했지만 후보작 | `1`점 |
| 없음                                                  | 오스카 언급 안 됨    | `0`점 |

In [23]:
def assign_award_score(row):
    if str(row["Winner"]).strip().lower() == "true":
        return 2
    elif str(row["Class"]).strip().lower() == "nomination":
        return 1
    else:
        return 1
    
df_oscars["award_score"] = df_oscars.apply(assign_award_score, axis=1)

In [24]:
df_oscars["award_score"].value_counts()

award_score
1    575
2    195
Name: count, dtype: int64

In [25]:
df_oscars["award_score"].value_counts(normalize=True)

award_score
1    0.746753
2    0.253247
Name: proportion, dtype: float64

# Top 10

In [26]:
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020-11-25,8.500,165
1,US,United States,Gabriel's Inferno,2020-05-29,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020-07-31,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020-11-19,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020-12-26,8.400,83


In [29]:
# 정렬 기준이 필요해요: vote_average or vote_count 기준
df_tmbd_top = df_tmbd_top.sort_values(["country", "vote_average", "vote_count"], ascending=[True, False, False])

# 기본 점수는 1.0
df_tmbd_top["top_score"] = 1.0

# 상위 20%: 나라별 상위 2편에게 +0.1점
df_tmbd_top["bonus_score"] = (
    df_tmbd_top.groupby("country").cumcount() < 2  # 상위 0, 1번 인덱스만
).astype(float) * 0.1

# 최종 점수
df_tmbd_top["top_score"] += df_tmbd_top["bonus_score"]

In [30]:
df_tmbd_top["title_lower"] = df_tmbd_top["title"].str.lower()

top_scores = df_tmbd_top.groupby("title_lower")["top_score"].sum().reset_index()

In [33]:
df_tmbd_top = df_tmbd_top.drop(columns=["bonus_score"])
df_tmbd_top

,country,country_name,title,release_date,vote_average,vote_count,title_lower,top_score
100,CN,China,Green Snake,2021-07-23,8.100,327,green snake,1.1
101,CN,China,New Gods: Nezha Reborn,2021-02-06,8.095,462,new gods: nezha reborn,1.1
325,CN,China,Ne Zha 2,2025-01-29,8.042,179,ne zha 2,1.0
220,CN,China,Hachiko,2023-03-31,7.879,253,hachiko,1.0
102,CN,China,B for Busy,2021-12-24,7.700,54,b for busy,1.0
...,...,...,...,...,...,...,...,...
306,US,United States,The Life of Chuck,2025-06-05,7.737,59,the life of chuck,1.0
305,US,United States,The Gorge,2025-02-13,7.711,2706,the gorge,1.0
307,US,United States,The Unbreakable Boy,2025-02-20,7.630,73,the unbreakable boy,1.0
308,US,United States,Sinners,2025-04-16,7.549,1563,sinners,1.0


In [36]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,success_imdb
0,Stepmom's Desire,2020-05-29,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,1
1,Pizza Dare 1,2020-04-10,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0
2,Bosomy Mom,2020-05-06,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0
3,Sonic the Hedgehog,2020-02-12,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,1
4,12,2020-09-04,movie,15.0,1.0,1.0,NaN,Salih Hashwan,unknown,SA,...,not there yet,"short film, short film",5.481,53,not there yet,1,0,1,21.863622,1


In [38]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner,award_score
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,NaN,1
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,NaN,1
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True,2
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,NaN,1
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,NaN,1


In [45]:
matched = df_tmbd_top["title"].isin(df_movie["title"])
print("매칭된 오스카 영화 수:", matched.sum(), "/", len(df_tmbd_top))

매칭된 오스카 영화 수: 318 / 333
